# setup translation with Etranslation API

This is an attempt to start performing translations using the Etranslation tool of the EC. I am working with Nuno Freire on this. The idea is that I make a request to Etransaltion and get the translation back fron the web app setup by Nuno

## References
-https://ec.europa.eu/cefdigital/wiki/pages/viewpage.action?pageId=216465916

-Mail from Nuno setting up th web app to receive the translations

The etranslation callback webapp is running.
You should use the callback URL and the error callback url, I believe. Use these URLs: 
callback URL: https://rnd-2.eanadev.org/etranslation-callback/callback
error callback URL: https://rnd-2.eanadev.org/etranslation-callback/error
 
When you send a request, the immediate response from eTranslation is a number (id of the request), and when the translated text is ready, etranslation sends it to the callback webapp running at rnd-2.
The webapp stores the translated text in files in this folder: /var/lib/etranslation/translations-repository
The files are named as follows:
<id_of_request>_<language_code>.txt
So, if you request a text to be translated into several languages, there will be several files with the same request id - one for each language.

Let me know if you have any questions.

Cheers,
Nuno


Here I attempt to create an ID for the translation

In [3]:
import requests
import json
import pandas as pd
from requests.auth import HTTPDigestAuth
import os
import os.path
import time

In [18]:
def receiveRequest(sourceLanguage,targetLanguage1,targetLanguage2,textToTranslate):
    
    eTranslationRestUrl = "https://webgate.ec.europa.eu/etranslation/si/translate"
    applicationName = 'Europeana_IR_20190225'
    password = "n2ux3PrvqQ5XpSad"

    translationRequest = {}
    translationRequest['sourceLanguage'] = f'{sourceLanguage}'
    #How can I request a text to be translated into several languages?
    # I think have to work with the fact that it is a list
    translationRequest['targetLanguages'] = [f'{targetLanguage1}',f'{targetLanguage2}']
    translationRequest['callerInformation'] = {"application" : applicationName, "username":applicationName}
    translationRequest['textToTranslate'] = f'{textToTranslate}'
    translationRequest['requesterCallback'] = 'https://rnd-2.eanadev.org/etranslation-callback/callback'
    translationRequest['requesterCallbackError']='https://rnd-2.eanadev.org/etranslation-callback/error'

    jsonTranslationRequest = json.dumps(translationRequest)

    jsonHeader = {'Content-Type' : 'application/json'}
    response = requests.post(eTranslationRestUrl, auth=HTTPDigestAuth(applicationName, password), headers = jsonHeader, data=jsonTranslationRequest)
    #Translation ID
    tr_ID=response.text
    #building the translation file that will be looked on the server
    #Upper case is needed because this is how the file is saved on Rnd2 server
    capital_source= f'{targetLanguage1}'.upper()
    file_ID= f'{tr_ID}_{capital_source}.txt'
    return file_ID

In [19]:
def ReceiveTranslation(file_ID):
    file_on_RnD2=f'../../../../../var/lib/etranslation/translations-repository/{file_ID}'
    # wait till file is created
    while not os.path.exists(file_on_RnD2):
         time.sleep(1) #wait in cycles of a second
    if os.path.isfile(file_on_RnD2):
        fileObject = open(file_on_RnD2, "r")
        data = fileObject.read()
        res_dict=json.loads(data) 
    else:
        raise ValueError("%s isn't a file!" % file_path)
    return res_dict

# Example translation one sentence

In [20]:
txt_to_translate =' How are you today?'
file_ID=receiveRequest('EN','IT','NL',txt_to_translate)
translated_txt=ReceiveTranslation(file_ID)
translated_txt

{'request-id': ['181118332'],
 'target-language': ['IT'],
 'translated-text': ['Come sei oggi?\n']}

# Example translation multiple sentences

Sending three different requests

In [168]:
lst_tr=['How are you',
'Eat well tonight',
'Sleep very well',]

In [169]:
#sending three translation requests
file_tr=[]
for i in lst_tr:
    file_ID=receiveRequest('en','it',i)
    file_tr.append(file_ID)

In [170]:
#receiving three translations
lst2=[]
for i in file_tr:
    trs=ReceiveTranslation(i)
    lst2.append(trs)

In [171]:
lst2

[{'request-id': ['179262573'],
  'target-language': ['IT'],
  'translated-text': ['Come siete\n']},
 {'request-id': ['179262574'],
  'target-language': ['IT'],
  'translated-text': ['Mangiare bene questa sera\n']},
 {'request-id': ['179262575'],
  'target-language': ['IT'],
  'translated-text': ['Dormire molto bene\n']}]

In [10]:
eTranslationlangdetection='https://language-tools.ec.europa.eu/NamedEntitiesWS/askLanguageDetection'

In [24]:
applicationName = 'Europeana_IR_20190225'
password = "n2ux3PrvqQ5XpSad"

In [ ]:
"credentials" : {
        "application" : "MY_APPLICATION",
        "password" : "my_password"
    },
    "format" : "docx",   
    "verbose" : "false"    
}

In [6]:
applicationName = 'Europeana_IR_20190225'
password = "n2ux3PrvqQ5XpSad"

In [16]:
langdetectrequest = {}
langdetectrequest['credentials'] = {"application" : 'Europeana_IR_20190225',"password" : "n2ux3PrvqQ5XpSad"}
langdetectrequest['format'] = "txt"
langdetectrequest['verbose'] = "true"
    
    
    # #How can I request a text to be translated into several languages?
    # # I think have to work with the fact that it is a list
    # translationRequest['targetLanguages'] = [f'{targetLanguage1}',f'{targetLanguage2}']
    # translationRequest['callerInformation'] = {"application" : applicationName, "username":applicationName}
    # translationRequest['textToTranslate'] = f'{textToTranslate}'
    # translationRequest['requesterCallback'] = 'https://rnd-2.eanadev.org/etranslation-callback/callback'
#translationRequest['requesterCallbackError']='https://rnd-2.eanadev.org/etranslation-callback/error'

In [17]:
langdetectrequest

{'credentials': {'application': 'Europeana_IR_20190225',
  'password': 'n2ux3PrvqQ5XpSad'},
 'format': 'txt',
 'verbose': 'true'}

In [13]:
# import requests
from requests.structures import CaseInsensitiveDict

# url = "https://reqbin.com/echo/post/json"
# headers = CaseInsensitiveDict()
# headers["Accept"] = "application/json"
# headers["Content-Type"] = "application/json"

# data = """
# {
#   "Id": 78912,
#   "Customer": "Jason Sweet",
# }
# """

# resp = requests.post(url, headers=headers, data=data)
# print(resp.status_code)

In [14]:
langdetectrequest = {}
headers = CaseInsensitiveDict()
headers['credentials'] = {"application" : 'Europeana_IR_20190225',"password" : "n2ux3PrvqQ5XpSad"}
headers['format'] = "txt"
headers['verbose'] = "true"
data = """
{
  "text"
}"""
response = requests.post(eTranslationlangdetection,headers=headers,data=data)

InvalidHeader: Value for header {credentials: {'application': 'Europeana_IR_20190225', 'password': 'n2ux3PrvqQ5XpSad'}} must be of type str or bytes, not <class 'dict'>

In [23]:
    
    eTranslationRestUrl = " https://language-tools.ec.europa.eu/NamedEntitiesWS/askLanguageDetection"
    applicationName = 'Europeana_IR_20190225'
    password = "n2ux3PrvqQ5XpSad"

    langdetectrequest = {}
    langdetectrequest = {}
    langdetectrequest['credentials'] = {"application" : 'Europeana_IR_20190225',"password" : "n2ux3PrvqQ5XpSad"}
    langdetectrequest['format'] = "txt"
    langdetectrequest['verbose'] = "true"
    langdetectrequest['verbose']
  

    jsonTranslationRequest = json.dumps(langdetectrequest)

    jsonHeader = {'Content-Type' : 'multipart/formdata'}
    response = requests.post(eTranslationRestUrl, auth=HTTPDigestAuth(applicationName, password), headers = jsonHeader, data=langdetectrequest)
    #Translation ID
    # tr_ID=response.text
    # #building the translation file that will be looked on the server
    # #Upper case is needed because this is how the file is saved on Rnd2 server
    # capital_source= f'{targetLanguage1}'.upper()
    # file_ID= f'{tr_ID}_{capital_source}.txt'
    # return file_ID

In [24]:
response

<Response [400]>